In [1]:
%matplotlib inline
from IPython import display
from matplotlib import pyplot as plt
from mxnet import autograd, nd
import random

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

In [3]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = nd.array(indices[i: min(i + batch_size, num_examples)])
        yield features.take(j), labels.take(j)

In [4]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, y)
    break


[[-0.71065867  0.905795  ]
 [ 0.57098913  0.42878667]
 [-0.3622695  -1.3198183 ]
 [ 2.4940155  -0.02706606]
 [ 0.6378637  -0.6909089 ]
 [-0.33887258 -0.76657826]
 [-0.2648481   0.0712087 ]
 [-1.3416548   2.0981746 ]
 [ 0.21350802 -0.87535167]
 [-0.50989187 -0.9891328 ]]
<NDArray 10x2 @cpu(0)> 
[-0.30060866  3.8858724   7.9475636   9.284722    7.8366117   6.1165957
  3.4211595  -5.6283774   7.589675    6.5313973 ]
<NDArray 10 @cpu(0)>


In [5]:
w = nd.random.normal(scale=0.01, shape=(num_inputs, 1))
b = nd.zeros(shape=(1,))
w, b

(
 [[ 0.01464221]
  [-0.01305814]]
 <NDArray 2x1 @cpu(0)>, 
 [0.]
 <NDArray 1 @cpu(0)>)

In [6]:
w.attach_grad()
b.attach_grad()

In [7]:
def linreg(X, w, b):
    return nd.dot(X, w) + b

In [8]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

In [9]:
def sgd(params, lr, batch_size):
    for param in params:
        print(param.grad)
        param[:] = param - lr * param.grad / batch_size

In [10]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        with autograd.record():
            l = loss(net(X, w, b), y)
        print(l)
        l.backward()
        sgd([w, b], lr, batch_size)
        break
    train_l = loss(net(features, w, b), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))
    break


[[2.6091337e+00]
 [1.5854513e+01]
 [8.2529411e+00]
 [1.4345195e+00]
 [9.9861031e+00]
 [5.2166119e+01]
 [5.1584381e-01]
 [3.2613724e+01]
 [1.1276247e+01]
 [1.2162410e-05]]
<NDArray 10x1 @cpu(0)>

[[ 8.325975]
 [32.032227]]
<NDArray 2x1 @cpu(0)>

[-42.20129]
<NDArray 1 @cpu(0)>
epoch 1, loss 15.906798


In [11]:
true_w, w

([2, -3.4], 
 [[-0.01033571]
  [-0.10915481]]
 <NDArray 2x1 @cpu(0)>)

In [12]:
x = nd.arange(4).reshape((4, 1))
x


[[0.]
 [1.]
 [2.]
 [3.]]
<NDArray 4x1 @cpu(0)>

In [13]:
x.attach_grad()

In [14]:
with autograd.record():
    y = 2 * x

In [15]:
y.backward()

In [16]:
x.grad


[[2.]
 [2.]
 [2.]
 [2.]]
<NDArray 4x1 @cpu(0)>